In [1]:
!wget http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/phix.fa

--2025-08-04 10:45:39--  http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/phix.fa
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 3.162.130.139, 3.162.130.139, 3.162.130.139, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|3.162.130.139|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5528 (5.4K) [application/octet-stream]
Saving to: ‘phix.fa.1’

phix.fa.1           100%[===================>]   5.40K  --.-KB/s    in 0s      

2025-08-04 10:45:39 (452 MB/s) - ‘phix.fa.1’ saved [5528/5528]



In [2]:
# This function is for reading a genome from a FASTA file.
def readGenome(filename):
    genome = ''
    # Open the file and read it line by line.
    with open(filename, 'r') as f:
        for line in f:
            # The header line in a FASTA file starts with '>', we want to ignore it.
            if not line.startswith('>'):
                # Add the sequence line to our genome string, removing any trailing whitespace.
                genome += line.rstrip()
    return genome

In [3]:
genome = readGenome('phix.fa.1')

In [4]:
# This is a simple, "naive" algorithm for finding a pattern 'p' within a text 't'.
def naive(p, t):
    # This list will store the starting positions of every match we find.
    occurrences = []
    # Loop through the text, but stop when the remaining text is shorter than the pattern.
    for i in range(len(t) - len(p) + 1):
        match = True # Assume we have a match until proven otherwise.
        # Loop through the pattern to check each character.
        for j in range(len(p)):
            # If a character doesn't match...
            if t[i+j] != p[j]:
                # ...it's not a match.
                match = False
                break # No need to check the rest of the pattern, so we break this inner loop.
        # If the 'match' variable is still True after the inner loop, we found a full match
        if match:
          # Record the starting position of the match.
          occurrences.append(i)
    return occurrences

In [5]:
t = 'AGCTTAGATAGC'
p = 'AG'
naive(p, t)

[0, 5, 9]

In [8]:
# This function simulates a DNA sequencer.
import random
def generateReads(genome, numReads, readLen):
    ''' Generate reads from random positions in the given genome. '''
    reads = []
    # Do this 'numReads' times.
    for _ in range(numReads):
        # Pick a random starting point in the genome.
        start = random.randint(0, len(genome)-readLen)
        # Cut out a piece of the genome of length 'readLen'.
        reads.append(genome[start : start+readLen])
    return reads

In [10]:
# EXAMPLE

# simulate generating 100 reads, each 100 bases long.
reads = generateReads(genome, 100, 100)
# Now, let's see how many of our simulated reads we can find in the original genome.
numMatched = 0
for r in reads:
 # Use our naive search function to find the read.
     matches = naive(r, genome)
     # If the list of matches is not empty, it means we found it.
     if len(matches) > 0:
         numMatched += 1

print('%d / %d reads matched the genome exactly!' % (numMatched, len(reads)))

100 / 100 reads matched the genome exactly!


In [11]:
!wget http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR266411_1.first1000.fastq

--2025-08-04 10:53:34--  http://d28rh4a8wq0iu5.cloudfront.net/ads1/data/ERR266411_1.first1000.fastq
Resolving d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)... 3.162.130.139, 3.162.130.139, 3.162.130.139, ...
Connecting to d28rh4a8wq0iu5.cloudfront.net (d28rh4a8wq0iu5.cloudfront.net)|3.162.130.139|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 254384 (248K) [audio/mpeg]
Saving to: ‘ERR266411_1.first1000.fastq.5’

ERR266411_1.first10 100%[===================>] 248.42K  --.-KB/s    in 0.001s  

2025-08-04 10:53:34 (426 MB/s) - ‘ERR266411_1.first1000.fastq.5’ saved [254384/254384]



In [12]:
# This function reads sequences from a FASTQ file.
def readFastq(filename):
    sequences = []
    with open(filename) as fh:
        # FASTQ files have 4 lines per sequence, so we loop through the file.
        while True:
            fh.readline() # 1. Skip the name line which starts with @
            seq = fh.readline().rstrip() # 2. This is the actual DNA sequence
            fh.readline() # 3. Skip the placeholder line, which just has a +
            fh.readline() # 4. Skip the base quality score line
            
            # If the sequence line is empty, we've reached the end of the file.
            if len(seq) == 0:
                break
            
            # Add the sequence to our list.
            sequences.append(seq)
    return sequences

In [14]:
# EXAMPLE

import collections
# Read the first 1000 reads from the specified FASTQ file.
phix_reads = readFastq('ERR266411_1.first1000.fastq')

# Create a Counter object to store the frequency of each base.
count = collections.Counter()
# Loop through every read we just loaded.
for read in phix_reads:
     # Update the counter with all the bases in the current read.
     count.update(read)
#
# Print the final counts of A, C, G, T, and any other characters (like N).
print(count)


Counter({'T': 30531, 'A': 28426, 'C': 21890, 'G': 19147, 'N': 6})


In [17]:
# EXAMPLE

# Let's count how many of our reads appear perfectly in the reference genome.
numMatched = 0
n = 0
for r in phix_reads:
    # Use our naive search function to find the read in the genome.
    matches = naive(r, genome)
    n += 1
    # If the list of matches is not empty, it means we found it.
    if len(matches) > 0:
        numMatched += 1
print('%d / %d reads matched the genome exactly!' % (numMatched, n))

7 / 1000 reads matched the genome exactly!


In [18]:
# Now let's try matching just the first 30 bases of each read
numMatched = 0
n = 0
for r in phix_reads:
    r = r[:30]  # just taking the first 30 bases
    matches = naive(r, genome)
    n += 1
    if len(matches) > 0:
        numMatched += 1
print('%d / %d reads matched the genome exactly!' % (numMatched, n))

459 / 1000 reads matched the genome exactly!


In [19]:
def reverseComplement(s):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A', 'N': 'N'}
    t = ''
    for base in s:
        t = complement[base] + t
    return t

In [20]:
numMatched = 0
n = 0
for r in phix_reads:
    r = r[:30]  # just taking the first 30 bases
    matches = naive(r, genome)
    matches.extend(naive(reverseComplement(r), genome))
    n += 1
    if len(matches) > 0:
        numMatched += 1
print('%d / %d reads matched the genome exactly!' % (numMatched, n))

932 / 1000 reads matched the genome exactly!
